In [23]:
# %% [code]
### DenseNet121 N_FFT 256 - 23 EPOCHS - 0.96 ACC ON 20% TEST

##############################################################################
import os, math
import numpy as np
seed = 2018
np.random.seed(seed)

import librosa
from scipy import signal

from matplotlib import pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.layers import Input

from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import CSVLogger

from keras import Model
from keras import backend as K

from keras.utils import to_categorical
from keras.preprocessing import image
 
from keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.mobilenet import MobileNet
from keras.applications.nasnet import NASNetLarge, NASNetMobile
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception

In [24]:
current_model = DenseNet121
#current_model = DenseNet169
#current_model = DenseNet201
#current_model = InceptionResNetV2
#current_model = InceptionV3
#current_model = MobileNet
#current_model = NASNetLarge
#current_model = NASNetMobile
#current_model = VGG16
#current_model = VGG19
#current_model = Xception

model_name = 'wingbeats_' + current_model.__name__

best_weights_path = model_name + '.weights.h5'
log_path = model_name + '.log'
monitor = 'val_acc'
batch_size = 32
epochs = 100
es_patience = 7
rlr_patience = 3

SR = 8000
N_FFT = 256
HOP_LEN = N_FFT // 6
input_shape = (129, 120, 1)

In [25]:
target_names = ['Ae. aegypti', 'Ae. albopictus', 'An. gambiae', 'An. arabiensis', 'C. pipiens', 'C. quinquefasciatus']

X_names = []
y = []
target_count = []

for i, target in enumerate(target_names):
    target_count.append(0)
    path = './Wingbeats/' + target + '/'
    for [root, dirs, files] in os.walk(path, topdown = False):
        for filename in files:
            name,ext = os.path.splitext(filename)
            if ext == '.wav':
                name = os.path.join(root, filename)
                y.append(i)
                X_names.append(name)
                target_count[i]+=1
                # if target_count[i] > 20000:
                #     break
    print (target, '#recs = ', target_count[i])

print ('total #recs = ', len(y))

X_names, y = shuffle(X_names, y, random_state = seed)
X_train, X_test, y_train, y_test = train_test_split(X_names, y, stratify = y, test_size = 0.20, random_state = seed)

print ('train #recs = ', len(X_train))
print ('test #recs = ', len(X_test))

Ae. aegypti #recs =  85553
Ae. albopictus #recs =  20231
An. gambiae #recs =  49471
An. arabiensis #recs =  19297
C. pipiens #recs =  30415
C. quinquefasciatus #recs =  74599
total #recs =  279566
train #recs =  223652
test #recs =  55914


In [26]:
from tensorflow.keras.preprocessing.image import apply_affine_transform

def shift(x, wshift, hshift, fill_mode='constant', cval=0.):
    """Desplaza una imagen ajustando automáticamente los ejes"""
    print(f"Forma de la imagen: {x.shape}")  # Para debug
    
    # Determinar automáticamente los ejes basado en la forma
    if len(x.shape) == 2:  # (alto, ancho)
        # Imagen en escala de grises sin canal
        row_axis, col_axis, channel_axis = 0, 1, None
    elif len(x.shape) == 3:  # (alto, ancho, canales) o (canales, alto, ancho)
        if x.shape[0] in [1, 3]:  # (canales, alto, ancho)
            row_axis, col_axis, channel_axis = 1, 2, 0
        else:  # (alto, ancho, canales)
            row_axis, col_axis, channel_axis = 0, 1, 2
    else:
        raise ValueError(f"Forma de imagen no soportada: {x.shape}")
    
    print(f"Ejes detectados - row: {row_axis}, col: {col_axis}, channel: {channel_axis}")
    
    tx = wshift * x.shape[col_axis]
    ty = hshift * x.shape[row_axis]
    
    if channel_axis is not None:
        x = apply_affine_transform(
            x, 
            tx=tx, 
            ty=ty, 
            channel_axis=channel_axis, 
            fill_mode=fill_mode, 
            cval=cval
        )
    else:
        # Para imágenes sin canal de color
        x = apply_affine_transform(
            x, 
            tx=tx, 
            ty=ty, 
            fill_mode=fill_mode, 
            cval=cval
        )
    
    return x
def random_data_shift(data, w_limit = (-0.25, 0.25), h_limit = (-0.0, 0.0), cval = 0., u = 0.5):
    if np.random.random() < u:
        wshift = np.random.uniform(w_limit[0], w_limit[1])
        hshift = np.random.uniform(h_limit[0], h_limit[1])
        data = shift(data, wshift, hshift, cval = cval)
    return data

# def random_data_shift(data, u = 0.5):
#     if np.random.random() < u:
#         data = np.roll(data, int(round(np.random.uniform(-(len(data)), (len(data))))))
#     return data

def train_generator():
    while True:
        for start in range(0, len(X_train), batch_size):
            x_batch = []
            y_batch = []
            
            end = min(start + batch_size, len(X_train))
            train_batch = X_train[start:end]
            labels_batch = y_train[start:end]
            
            for i in range(len(train_batch)):
                data, rate = librosa.load(train_batch[i], sr = SR)

                #data = random_data_shift(data, u = 1.0)

                data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
                data = librosa.amplitude_to_db(data)

                data = np.flipud(data)

                data = np.expand_dims(data, axis = -1)
                data = random_data_shift(data, w_limit = (-0.25, 0.25), h_limit = (-0.0, 0.0), cval = np.min(data), u = 1.0)

                # data = np.squeeze(data, axis = -1)
                # plt.imshow(data, cmap = 'gray')
                # plt.show()
                # data = np.expand_dims(data, axis = -1)

                x_batch.append(data)
                y_batch.append(labels_batch[i])

            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            
            y_batch = to_categorical(y_batch, len(target_names))
            
            yield x_batch, y_batch

def valid_generator():
    while True:
        for start in range(0, len(X_test), batch_size):
            x_batch = []
            y_batch = []
            
            end = min(start + batch_size, len(X_test))
            test_batch = X_test[start:end]
            labels_batch = y_test[start:end]
            
            for i in range(len(test_batch)):
                data, rate = librosa.load(test_batch[i], sr = SR)

                data = librosa.stft(data, n_fft = N_FFT, hop_length = HOP_LEN)
                data = librosa.amplitude_to_db(data)

                data = np.flipud(data)

                data = np.expand_dims(data, axis = -1)

                x_batch.append(data)
                y_batch.append(labels_batch[i])

            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            
            y_batch = to_categorical(y_batch, len(target_names))
            
            yield x_batch, y_batch

In [27]:
img_input = Input(shape = input_shape)

model = current_model(input_tensor = img_input, classes = len(target_names), weights = None)

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])  

callbacks_list = [ModelCheckpoint(monitor = monitor,
                                filepath = best_weights_path, 
                                save_best_only = True, 
                                save_weights_only = True,
                                verbose = 1), 
                    EarlyStopping(monitor = monitor,
                                patience = es_patience, 
                                verbose = 1),

                    CSVLogger(filename = log_path)]

model.fit(train_generator(),
    steps_per_epoch = int(math.ceil(float(len(X_train)) / float(batch_size))),
    validation_data = valid_generator(),
    validation_steps = int(math.ceil(float(len(X_test)) / float(batch_size))),
    epochs = epochs,
    callbacks = callbacks_list,
    shuffle = False)

model.load_weights(best_weights_path)

loss, acc = model.evaluate_generator(valid_generator(),
        steps = int(math.ceil(float(len(X_test)) / float(batch_size))))

#print('loss:', loss)
print('Test accuracy:', acc)

Forma de la imagen: (129, 120, 1)
Ejes detectados - row: 0, col: 1, channel: 2


/tmp/ipykernel_2943272/3682913099.py:72: UserWarning: amplitude_to_db was called on complex input so phase information will be discarded. To suppress this warning, call amplitude_to_db(np.abs(S)) instead.
  data = librosa.amplitude_to_db(data)


ValueError: 'row_axis', 'col_axis', and 'channel_axis' must be distinct